In [1]:
from pyspark import SparkConf, SparkContext
from glob import glob
import sys
import json
import io
from tqdm import tqdm
import bz2

REDDIT_COMMENTS_DIR = '/mnt/ceph/storage/corpora/corpora-thirdparty/corpus-reddit-public-comments/'
DATA_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/'
SUBREDDIT = 'tipofmytongue'

conf=SparkConf()
conf.setAll([("spark.executor.memory", '20g'), ("spark.driver.memory", '20g')])
sc = SparkContext(master="yarn",appName="reddit-tomt",conf=conf)
sc

<SparkContext master=yarn appName=reddit-tomt>

In [24]:
def load_zstd_tomt_entries_from_file(file_name):
    zstandard_lib_path = f'{DATA_DIR}reddit-tomt/virtual-env/lib/python3.8/site-packages/'
    if zstandard_lib_path not in sys.path:
        sys.path.insert(0, zstandard_lib_path)
    import zstandard

    with open(file_name,"rb") as inf:
        dctx = zstandard.ZstdDecompressor(max_window_size=2147483648)
        stream_reader = dctx.stream_reader(inf, read_size=16384)
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        for idx, line in enumerate(tqdm(text_stream)):
            try:
                record = json.loads(line)
                #yield record
                if record['subreddit'].lower() == SUBREDDIT and 'body' in record and record['body']:
                    yield line
            except:
                pass


In [25]:
for i in load_zstd_tomt_entries_from_file(REDDIT_COMMENTS_DIR + 'RC_2014-12.zst'):
    print(i)
    break

0it [00:00, ?it/s]Exception ignored in: <generator object load_zstd_tomt_entries_from_file at 0x7faa02ba30b0>
Traceback (most recent call last):
  File "/tmp/ipykernel_102/1593679532.py", line 3, in <module>
RuntimeError: generator ignored GeneratorExit
2365it [00:00, 44776.35it/s]

{"edited":false,"archived":false,"name":"t1_cmheoej","body":"Damn it. First one I know the answer to and I'm 3 hours late :(","author":"UJ95x","score_hidden":false,"created_utc":"1417392090","subreddit_id":"t5_2r4oc","link_id":"t3_2nvbc2","id":"cmheoej","distinguished":null,"downs":0,"gilded":0,"author_flair_text":null,"parent_id":"t3_2nvbc2","ups":17,"controversiality":0,"subreddit":"tipofmytongue","score":17,"retrieved_on":1425748654,"author_flair_css_class":null}



In [26]:
sc.parallelize(glob(f'{REDDIT_COMMENTS_DIR}/*.zst'))\
    .flatMap(lambda i: load_zstd_tomt_entries_from_file(i))\
    .repartition(100)\
    .saveAsTextFile('reddit-tomt/zstd-comments-files', compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")